In [4]:
from pydrake.all import (
    DiagramBuilder, MultibodyPlant, SceneGraph,
    Parser, Simulator, Meshcat, MeshcatVisualizerCpp
)

meshcat = Meshcat()  # starts a Meshcat server

builder = DiagramBuilder()
plant, scene_graph = MultibodyPlant(time_step=0.0), SceneGraph()
plant.RegisterAsSourceForSceneGraph(scene_graph)

parser = Parser(plant)
parser.AddModels("your_model.sdf")   # <- your SDF here

plant.Finalize()

MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()

context = diagram.CreateDefaultContext()
simulator = Simulator(diagram, context)
simulator.Initialize()
simulator.AdvanceTo(0.1)   # let geometry show up


ImportError: cannot import name 'MeshcatVisualizerCpp' from 'pydrake.all' (/usr/local/lib/python3.12/dist-packages/pydrake/all.py)

In [ ]:
<?xml version="1.0"?>
<sdf version="1.7">
  <model name="solo_like_cup">
    <link name="base_link">
      <inertial>
        <pose>0 0 0.06 0 0 0</pose>
        <mass>0.03</mass>
        <inertia>
          <ixx>0.00015</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.00015</iyy>
          <iyz>0</iyz>
          <izz>0.00025</izz>
        </inertia>
      </inertial>

      <!-- visuals (same as before, you can change color to red if you want) -->
      <visual name="visual_cup_bottom">
        <pose>0 0 0.015 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.035</radius>
            <length>0.03</length>
          </cylinder>
        </geometry>
        <material>
          <diffuse>1.0 0.0 0.0 1.0</diffuse>
        </material>
      </visual>

      <visual name="visual_cup_middle">
        <pose>0 0 0.045 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.045</radius>
            <length>0.03</length>
          </cylinder>
        </geometry>
        <material>
          <diffuse>1.0 0.0 0.0 1.0</diffuse>
        </material>
      </visual>

      <visual name="visual_cup_top">
        <pose>0 0 0.075 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.055</radius>
            <length>0.03</length>
          </cylinder>
        </geometry>
        <material>
          <diffuse>1.0 0.0 0.0 1.0</diffuse>
        </material>
      </visual>

      <visual name="visual_cup_base_disk">
        <pose>0 0 0.005 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.03</radius>
            <length>0.002</length>
          </cylinder>
        </geometry>
        <material>
          <diffuse>1.0 0.0 0.0 1.0</diffuse>
        </material>
      </visual>

      <!-- collision body -->
      <collision name="collision_cup_body">
        <pose>0 0 0.055 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.045</radius>
            <length>0.11</length>
          </cylinder>
        </geometry>
      </collision>
    </link>
  </model>
</sdf>


In [ ]:
#!/usr/bin/env python3

from pydrake.all import (
    DiagramBuilder, MultibodyPlant, SceneGraph,
    Parser, RigidTransform, RollPitchYaw, Simulator,
    Meshcat, MeshcatVisualizerCpp
)

def main():
    meshcat = Meshcat()  # will print the URL to open in browser

    builder = DiagramBuilder()
    plant, scene_graph = MultibodyPlant(time_step=0.0), SceneGraph()
    plant.RegisterAsSourceForSceneGraph(scene_graph)

    parser = Parser(plant)

    # Add ground (optional, but nice reference)
    parser.AddModelsFromUrl("package://drake/systems/plant/test/ground.sdf")

    # Load the cup model (path relative to where you run the script)
    cup_model_path = "solo_like_cup.sdf"
    cup_model = parser.AddModels(cup_model_path)[0]

    # Basic geometry numbers
    base_z = 0.0        # bottom on ground
    cup_height = 0.11   # approx visual height
    dx = 0.08           # horizontal spacing for the base cups
    vertical_offset = 0.02  # gap between the top and bottom row

    # Positions of cup bases (world frame)
    bottom_left_pos = [ -dx, 0.0, base_z ]
    bottom_right_pos = [  dx, 0.0, base_z ]
    top_pos = [
        0.0,
        0.0,
        base_z + cup_height + vertical_offset
    ]

    # Add three instances of the cup
    # Note: With Drake's Parser.AddModels, we get a "model instance" id;
    # use plant.SetDefaultFreeBodyPose or WeldFrames to place them.

    plant.Finalize()

    # Once plant is finalized, we can set default poses.
    # Find the body for the cup (the base link)
    body = plant.GetBodyByName("base_link", cup_model)

    # bottom left
    plant.SetDefaultFreeBodyPose(
        body,
        RigidTransform(RollPitchYaw(0, 0, 0), bottom_left_pos)
    )

    # For the other two cups, we want separate instances.
    # Easiest is to add 3 separate model instances before Finalize.
    # So: Let's actually do it the "right" way and rebuild:

if __name__ == "__main__":
    # Below is a corrected version that creates three distinct model instances.

    meshcat = Meshcat()

    builder = DiagramBuilder()
    plant, scene_graph = MultibodyPlant(time_step=0.0), SceneGraph()
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)

    parser.AddModelsFromUrl("package://drake/systems/plant/test/ground.sdf")

    cup_model_path = "solo_like_cup.sdf"

    # Create three model instances
    cup1 = parser.AddModels(cup_model_path)[0]
    cup2 = parser.AddModels(cup_model_path)[0]
    cup3 = parser.AddModels(cup_model_path)[0]

    plant.Finalize()

    # Basic geometry
    base_z = 0.0
    cup_height = 0.11
    dx = 0.08
    vertical_offset = 0.02

    bottom_left_pos = [ -dx, 0.0, base_z ]
    bottom_right_pos = [  dx, 0.0, base_z ]
    top_pos = [
        0.0,
        0.0,
        base_z + cup_height + vertical_offset
    ]

    # Set default poses for each instance
    body1 = plant.GetBodyByName("base_link", cup1)
    body2 = plant.GetBodyByName("base_link", cup2)
    body3 = plant.GetBodyByName("base_link", cup3)

    plant.SetDefaultFreeBodyPose(
        body1, RigidTransform(RollPitchYaw(0, 0, 0), bottom_left_pos)
    )
    plant.SetDefaultFreeBodyPose(
        body2, RigidTransform(RollPitchYaw(0, 0, 0), bottom_right_pos)
    )
    plant.SetDefaultFreeBodyPose(
        body3, RigidTransform(RollPitchYaw(0, 0, 0), top_pos)
    )

    MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    simulator = Simulator(diagram, context)
    simulator.Initialize()
    simulator.AdvanceTo(0.1)
